# Ranker

In [ ]:
RANKER_PROMPT = """You are a highly critical and discerning research program evaluator with deep expertise in compression and the sciences.

There has been a recent breakthrough in the field of compression. Your task is to explore new research discoveries that this breakthrough enables.

Your task is to evaluate the paper based on the following criteria:

You are going to be presented with another research paper



**Breakthrough Summary (LMCompress):**
The paper "Understanding is Compression" introduces LMCompress, a lossless compression framework using LLMs as data understanding engines. It tokenizes data, uses a generative LLM to estimate token probabilities, and applies arithmetic coding. By fine-tuning LLMs on specific domains, it significantly outperforms traditional methods (JPEG-XL, FLAC, H.264, bz2), suggesting a shift to a "Kolmogorov paradigm" where understanding and modeling the source, not just redundancy removal, is key. This has major implications for fields like 6G communication by mitigating bandwidth limits through deeper data understanding.

Your task is to critically evaluate each proposed research idea against the following specific metrics. For each idea, provide a structured evaluation:

**Evaluation Metrics:**

1.  **NOVELTY (Score 1-5, 5 being highly novel):**
    *   How original is this idea?
    *   Does it propose a genuinely new application, methodology, or theoretical extension of the LMCompress breakthrough?
    *   Or, is it an obvious next step, a minor tweak to existing work, or a rehash of old concepts merely relabeled with "LLM compression"?
    *   **Justification required.**

2.  **IMPORTANCE (Score 1-5, 5 being critically important):**
    *   What is the potential impact of this idea if successfully realized?
    *   Would it solve a significant problem, open up major new capabilities, or substantially advance our understanding or application of semantic compression?
    *   Consider its practical, theoretical, or societal relevance.
    *   **Justification required.**

3.  **FEASIBILITY & VERIFIABILITY (Combined Score 1-5, 5 being highly feasible & verifiable):**
    *   **Feasibility:** Can a proof-of-concept (PoC) or a minimal viable demonstration for this idea be reasonably implemented with current or near-future resources and technology (assuming the LMCompress capability exists)?
    *   **Verifiability:** Can the success or failure of this idea be clearly demonstrated and validated *quickly*?
        *   This means an output that can be checked by code (e.g., a compression ratio achieved, a task completed, a specific measurable outcome).
        *   OR, by a human inspecting a tangible result (e.g., a decoded image's quality, a summarized text's accuracy, a correctly derived mathematical step, a functioning compressed communication channel) within minutes.
        *   Is the proposed verification method robust and unambiguous?
    *   **Justification required.**

4.  **RELIANCE ON SEMANTIC COMPRESSION (Score 1-5, 5 meaning critically reliant):**
    *   Is the *efficient compression of semantic data* (as enabled by LMCompress) the *core* enabler or the *primary bottleneck* that this idea addresses?
    *   In other words, was this idea largely impractical, significantly less effective, or even impossible *before* a breakthrough like LMCompress, specifically due to the inability to efficiently compress or leverage the semantic understanding of data like text, images, audio, or complex symbolic structures?
    *   If the idea could have been pursued almost as effectively with older compression techniques or non-compression-focused AI, it scores lower here.
    *   **Justification required.**

**Overall Assessment:**
Based on the individual scores, provide a brief overall recommendation for each idea (e.g., "Highly Promising," "Worth Exploring," "Needs Refinement," "Low Priority").

**Input Format:**
You will be given a list of research ideas.

**Output Format:**
For each idea, provide:
Idea #X: [Name of Idea, if provided, or a brief title you assign]
-   Novelty: [Score]/5. Justification: [...]
-   Importance: [Score]/5. Justification: [...]
-   Feasibility & Verifiability: [Score]/5. Justification: [...]
-   Reliance on Semantic Compression: [Score]/5. Justification: [...]
-   Overall Assessment: [...]
---

Please proceed to evaluate the ideas provided.
"""